In [ ]:
import pandas as pd
import glob
# This functions are to plot the graphs
import matplotlib.pyplot as plt
import seaborn as sns
# By here
# This functions are to do mapping
import numpy as np
import json
import folium
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
# By here

%matplotlib inline

import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')
#### The above code is to set up the font and validate the KOREAN

# This is to test to call the excel files
glob.glob('D:\\OneDrive-Jongki\\OneDrive - Illinois Institute of Technology\\Jongki-study\\05_Manual\\17_Python\\DataScience\\data\\Downloaded\\지역*.xls')

In [ ]:
# This is to call the excel filess
station_files = glob.glob('D:\\OneDrive-Jongki\\OneDrive - Illinois Institute of Technology\\Jongki-study\\05_Manual\\17_Python\\DataScience\\data\\Downloaded\\지역*.xls')
print(station_files)

In [ ]:
# This work is to merge all excel files in 'station_files'
# Need to install 'xlrd'
tmp_raw = []
for file_name in station_files:
    # This is to skip the first two rows because they are nothing
    tmp = pd.read_excel(file_name, header=2)
    tmp_raw.append(tmp)

station_raw = pd.concat(tmp_raw)
print(station_raw)

In [ ]:
print(station_raw.info())

In [ ]:
print(station_raw)

In [ ]:
# This is to re-organize the data, 'station', having only few items to meet the purpose of this study
station = pd.DataFrame({'Store':station_raw['상호'], 'Address':station_raw['주소'],'Price':station_raw['휘발유'],'Self':station_raw['셀프여부'],'Brand':station_raw['상표']})
print(station.head())

In [ ]:
# This is to make a column for '구'
# For this, we use 'eachAddress.split()' function
# Among 'Address' which is in 'station', we split the words which is distinguishable by space
station['구'] = [eachAddress.split()[1] for eachAddress in station['Address']]
print(station.head())

In [ ]:
# Since we have a big data set, we need to check a data whether there is a duplicate name or a unwanted subject
station['구'].unique()

In [ ]:
# This is how to filter the unwanted subeject
# Let's assume there is '1' in front of '서울특별시' which makes a return as '서울특별시' when we run 'station['구'] = [eachAddress.split()[1] for eachAddress in station['Address']]'
# station[station['구]=='서울특별시']

# AND THEN
# station.loc[station['구]=='서울특별시', '구'] = '성동구'
# station['구'].unique

In [ ]:
# This is to rename '구' to 'District'
station.rename(columns={station.columns[5]:'District'}, inplace=True)
print(station.head())

In [ ]:
# IF there is a store doesn't report a price and they set as '-'
# station[station['Price']=='-']

# AND THEN, it is to exclude the data without price information
# station = staion[station['Price'] != '-']
# station.head()

In [ ]:
# This is to replace the data type in 'Price' to 'float'
station['Price'] = [float(value) for value in station['Price']]

# This is to reset the index because it may contain redundant items 
station.reset_index(inplace=True)
del station['index']
station.info()

In [ ]:
station.boxplot(column='Price', by='Self', figsize=(10,6))

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='Brand', y='Price', hue='Self', data=station, palette='Set3')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='Brand', y='Price', data=station, palette='Set3')
sns.swarmplot(x='Brand', y='Price', data=station, color='.6')
plt.show()

In [ ]:
print(station.sort_values(by='Price', ascending=False))

In [ ]:
gu_data = pd.pivot_table(station, index=['District'], values=['Price'], aggfunc=np.mean)
print(gu_data)

In [ ]:
geo_path = 'D:\\OneDrive-Jongki\\OneDrive - Illinois Institute of Technology\\Jongki-study\\05_Manual\\17_Python\\DataScience\\data\\02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [ ]:
map = folium.Map(loaction=[37.5502, 126.982], zoom_start=100)
map.choropleth(geo_data = geo_str, data = gu_data['Price'], columns=[gu_data.index, gu_data['Price']], key_on='feature.id')
map